### Swarm set up

In [1]:
from swarm import Swarm, Agent

client = Swarm()

### Basic use

In [2]:
agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
)

In [3]:
messages = [{"role": "user", "content": "What is the capital of Portugal"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

The capital of Portugal is Lisbon.


### Handoff to another agent

In [4]:
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
)

spanish_agent = Agent(
    name="Spanish Agent",
    instructions="You only speak Spanish.",
)


def transfer_to_spanish_agent():
    """Transfer spanish speaking users immediately."""
    return spanish_agent


english_agent.functions.append(transfer_to_spanish_agent)



In [5]:
messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

Hola, estoy bien, gracias. ¿Y tú, cómo estás?


In [6]:
messages = [{"role": "user", "content": "Hi. How are you?"}]
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?


### Add a tool

In [7]:
import wikipedia

def wikipedia_lookup(q: str) -> str:
    """Look up a query in Wikipedia and return the result"""
    return wikipedia.page(q).summary

In [19]:
wikiagent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that finds information from Wikipedia.
    Work out what the user is interested in, look up the relevant 
    information and from that provide an answer.""",
    functions=[wikipedia_lookup],
)

In [9]:
messages = [{"role": "user", "content": "What is the population of the capital of Spain"}]

response = client.run(agent=wikiagent, messages=messages)
print(response.messages[-1]["content"])

The population of Madrid, the capital of Spain, is almost 3.4 million inhabitants within the city itself, and the metropolitan area has a population of approximately 7 million.


But this often fails because it tries to look up pages that do not exist. A more robust approach is necessary.

In [36]:
wikiagent = Agent(
    name="Agent",
    instructions="""You are a helpful agent that finds information from Wikipedia.
    Work out what the user is interested in.
    If possible pick a keyword, look up the relevant 
    information for that keyword.
    If you arte able provided an answer from that information.
    Otherwise, try again with another query.
    """,
    functions=[wikipedia_lookup],
)

In [39]:
messages = [{"role": "user", "content": "What is the population of the largest capital in the EU"}]

response = client.run(agent=wikiagent, messages=messages)
print(response.messages[-1]["content"])

The largest capital in the EU by population within the city boundary is Berlin. If you need the exact population figure or more detailed information, please let me know!


In [40]:
print(response.messages)

[{'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_i63hGfOPVZ5MpWvJS9HAI3Iu', 'function': {'arguments': '{"q":"largest capital in the EU by population"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}], 'sender': 'Agent'}, {'role': 'tool', 'tool_call_id': 'call_i63hGfOPVZ5MpWvJS9HAI3Iu', 'tool_name': 'wikipedia_lookup', 'content': 'This is a list of the largest cities in the European Union according to the population within their city boundary. The cities listed all have populations over 300,000. The list deals exclusively with the areas within city administrative boundaries as opposed to urban areas or larger urban zones (metropolitan areas), which are generally larger in terms of population than the main city (although they can also be smaller, in some of the cases presented).\nAs some cities have a very narrow boundary and others a very wide one, the list may not give an accurate view of the comparative magni

In [37]:
messages = [{"role": "user", "content": "Which is bigger Paris or Berlin"}]

response = client.run(agent=wikiagent, messages=messages)
print(response.messages[-1]["content"])

In terms of population and area, Berlin is bigger than Paris.

- **Berlin**: As the capital and largest city of Germany, Berlin has more than 3.85 million inhabitants within the city limits and covers an extensive area. The greater Berlin urban area has a population of over 4.5 million.

- **Paris**: As the capital and largest city of France, Paris has a population of about 2.1 million within the city area of around 105 km².

Berlin is larger both in terms of population and urban area as compared to Paris.


In [38]:
for m in response.messages:
    print(m)

{'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_xvKmPqQEbRr1N53SB28YL3YL', 'function': {'arguments': '{"q": "Paris"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}, {'id': 'call_T5OdF77NpmvnWUXnTxQeon4M', 'function': {'arguments': '{"q": "Berlin"}', 'name': 'wikipedia_lookup'}, 'type': 'function'}], 'sender': 'Agent'}
{'role': 'tool', 'tool_call_id': 'call_xvKmPqQEbRr1N53SB28YL3YL', 'tool_name': 'wikipedia_lookup', 'content': "Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an official estimated population of 2,102,650 residents in January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts